In [1]:
#Getting all danish movie between 2020/01/01 and 2024/12/31
import os
import requests
import pandas as pd
from dotenv import load_dotenv
from time import sleep 

load_dotenv()
tmdb_api_token = os.getenv("TMDB_API_TOKEN")


base_url = (
    "https://api.themoviedb.org/3/discover/movie"
    "?include_adult=false"
    "&include_video=false"
    "&language=en-US"
    "&sort_by=popularity.desc"
    "&with_original_language=da"
    "&primary_release_date.gte=2020-01-01" #Only movies after
    "&primary_release_date.lte=2024-12-31" #Only movies before  
)

headers = {
    "accept": "application/json",
    "Authorization": f"Bearer {tmdb_api_token}"
}

all_results = []

# Loop through all pages (adjust max page if needed)
for page in range(1, 289):
    url = f"{base_url}&page={page}"
    resp = requests.get(url, headers=headers).json()

    # Stop if we reach the last page
    if "results" not in resp or not resp["results"]:
        break

    all_results.extend(resp["results"])
    sleep(0.02)  #Just to be sure we don't request to many requests
df_movies = pd.DataFrame(all_results)

print(f"Total movies collected: {len(df_movies)}")
df_movies.to_csv("danish_movies_upto_2024.csv", index=False)


Total movies collected: 931


In [2]:
#Running through all the movies and getting actors id, name and gender
load_dotenv()
tmdb_api_token = os.getenv("TMDB_API_TOKEN")

headers = {
    "accept": "application/json",
    "Authorization": f"Bearer {tmdb_api_token}"
}


movie_ids = df_movies["id"].tolist()

all_credits = []   

for movie_id in movie_ids:
    url = f"https://api.themoviedb.org/3/movie/{movie_id}/credits?language=en-US"
    resp = requests.get(url, headers=headers).json()

    
    for member in resp.get("cast", []):
        all_credits.append({
            "movie_id": movie_id,
            "actor_id": member["id"],
            "actor_name": member["name"],
            "gender": member.get("gender")
        })

   
    sleep(0.02) #Just to be sure we don't request to many requests


df_cast = pd.DataFrame(all_credits)

print(f"Credits rows collected: {len(df_cast)}")
df_cast.to_csv("danish_movies_cast_credits.csv", index=False)

Credits rows collected: 4767


In [4]:
#Running through all the movies and getting actors id, name and gender
from collections import defaultdict

load_dotenv()
tmdb_api_token = os.getenv("TMDB_API_TOKEN")

headers = {
    "accept": "application/json",
    "Authorization": f"Bearer {tmdb_api_token}"
}


movie_ids = df_movies["id"].tolist()


actor_movies = defaultdict(lambda: {"actor_name": "", "movie_ids": set(), "gender": None})

for movie_id in movie_ids:
    url = f"https://api.themoviedb.org/3/movie/{movie_id}/credits?language=en-US"
    resp = requests.get(url, headers=headers).json()

    for member in resp.get("cast", []):
        actor_id = member["id"]
        actor_movies[actor_id]["actor_name"] = member["name"]
        actor_movies[actor_id]["gender"] = member.get("gender")
        actor_movies[actor_id]["movie_ids"].add(movie_id)

    sleep(0.02)  

rows = []
for actor_id, info in actor_movies.items():
    rows.append({
        "actor_id": actor_id,
        "actor_name": info["actor_name"],
        "gender": info["gender"],
        "movie_ids": list(info["movie_ids"])  
    })

df_actors = pd.DataFrame(rows)

print(f"Total unique actors collected: {len(df_actors)}")
df_actors.to_csv("danish_actors_movies_2024.csv", index=False)

Total unique actors collected: 3131


In [ ]:
#Getting birthday and deathday for the actors
headers = {
    "accept": "application/json",
    "Authorization": f"Bearer {tmdb_api_token}"
}

actor_ids = df_actors["actor_id"].tolist()
actors_bd_list = []

for actor_id in actor_ids:
    url = f"https://api.themoviedb.org/3/person/{actor_id}"
    resp = requests.get(url, headers=headers).json()

    birthday = resp.get("birthday", None)
    deathday = resp.get("deathday", None)
    
    actors_bd_list.append({
        "actor_id": actor_id,
        "birthday": birthday,
        "deathday": deathday
    })
    
    sleep(0.02)  

df_actorsbd = pd.DataFrame(actors_bd_list)

df_actorsbd = df_actorsbd.merge(df_actors[["actor_id", "actor_name"]], on="actor_id")

df_actorsbd.to_csv("danish_actors_bd_dd_2024.csv", index=False)

print(df_actorsbd.head())